In [ ]:
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.2 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login, HfApi
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
api = HfApi()

In [ ]:
notebook_login()

In [ ]:
#Run if you want to create new repo
api.create_repo(repo_id="model_demo")

RepoUrl('https://huggingface.co/pzalavad/model_demo', endpoint='https://huggingface.co', repo_type='model', repo_id='pzalavad/model_demo')

In [ ]:
folder_path = '/content/drive/MyDrive/TweetData/SecondTryModel'

In [ ]:
files = os.listdir(folder_path)
for file_name in files:
  api.upload_file(
      path_or_fileobj=os.path.join(folder_path, file_name),
      path_in_repo=file_name,
      repo_id="pzalavad/model_demo",
      repo_type="model",
  )

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

# Test Deployed model with preprocessing steps

In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 11.7 MB/s eta 0:00:00


In [ ]:
import nltk
import urllib
import bs4 as bs
import re
import contractions
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string
from gensim.models import Word2Vec

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
def remove_stopwords(sentences):
    stop_words = set(stopwords.words('english'))
    filtered_sentences = []

    for sentence in sentences:
      words = word_tokenize(sentence)
      temp = [word for word in words if word not in stop_words]
      filtered_sent = ' '.join(temp)
      filtered_sentences.append(filtered_sent)

    return filtered_sentences

def remove_punctuation(sentences):
    punctuation = string.punctuation
    filtered_sentences = []

    for sentence in sentences:
      temp = [char for char in sentence if char not in punctuation]
      new_sent = ''.join(temp)
      filtered_sentences.append(new_sent)
    return filtered_sentences

def stem_sentences(sentences):
    stemmer = PorterStemmer()
    stemmed_sentences = []

    for sentence in sentences:
        words = word_tokenize(sentence)
        temp = [stemmer.stem(word) for word in words]
        stemmed_sent = ' '.join(temp)
        stemmed_sentences.append(stemmed_sent)
    return stemmed_sentences

def lem_sentences(sentences):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentences = []

    for sentence in sentences:
        words = word_tokenize(sentence)
        temp = [lemmatizer.lemmatize(word) for word in words]
        stemmed_sent = ' '.join(temp)
        lemmatized_sentences.append(stemmed_sent)

    return lemmatized_sentences
def preprocess_sentence(sentences):
    # Remove punctuation
    sentences = remove_punctuation(sentences)
    sentences = remove_stopwords(sentences)
    sentences = stem_sentences(sentences)
    sentences = lem_sentences(sentences)
    # WRITE YOUR CODE HERE #
    return sentences
def clean_content(text):

  # remove twitter handles
  clean_text = re.sub(r'@\w+\s?', "", str(text))

  # convert to lowercase
  clean_text = clean_text.lower()

  # remove links http:// or https://
  clean_text = re.sub(r'https?:\/\/\S+', '', clean_text)

  # remove links beginning with www. and ending with .com
  clean_text = re.sub(r'www\.[a-z]?\.?(com)+|[a-z]+\.(com)', '', clean_text)

  # remove html reference characters
  clean_text = re.sub(r'&[a-z]+;', '', clean_text)

  # remove non-letter characters besides spaces "/", ";" "[", "]" "=", "#" Regex syntax
  clean_text = re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', clean_text)

  clean_text = contractions.fix(clean_text)
  sentences = sent_tokenize(clean_text)
  filtered_sentences = preprocess_sentence(sentences)
  clean_text = '.'.join(filtered_sentences)
  return clean_text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
test_tweets = {"I can't believe how disrespectful some people can be! It's infuriating! #Angry #Frustrated #Outrage": 'anger',
          "Just had the worst customer service experience ever! Feeling furious right now. #AngryCustomer #UnbelievableService": 'anger',
          "I am so lucky to have found my soulmate. Every day with you feels like a dream come true. #Love #Grateful #Blessed": 'love',
          "Love is the most beautiful feeling in the world. It makes everything brighter and better. Spread love everywhere you go. #SpreadLove #LoveIsAllYouNeed": 'love',
          "Just finished reading an interesting book. It was a thought-provoking journey. 📚 #BookLover #Thoughtful #Recommendation": 'neutral',
          "The weather today is neither too hot nor too cold. Just perfect for a walk in the park. 🌤️ #WeatherUpdate #EnjoyingNature": 'neutral',
          "I woke up feeling incredibly motivated and ready to conquer the day! Let's make it a productive one, folks! 💪 #Motivated #PositiveVibes #GoGetIt": 'positive',
          "Received some great news today! My hard work finally paid off. Feeling on top of the world right now. 🌟 #AchievementUnlocked #HappyDance": 'positive',
          "Feeling heartbroken today. It's hard to let go of something you held so dear. 💔 #Heartbroken #Emotional #Sadness": 'sadness',
          "Sometimes life throws curveballs, and it's okay to feel sad. Just remember to take it one step at a time and lean on your loved ones for support. 🌸 #SadButStrong #SupportSystem": 'sadness',
          "OMG! I just got the most amazing surprise from my friends! They truly know how to make me feel special. 🎉 #Surprised #Blessed #FriendsForever": 'surprise',
          "Didn't expect to find a hidden gem of a restaurant in this small town. The food blew my mind! Highly recommend checking it out. 🍽️ #HiddenGem #FoodieDiscoveries": 'surprise',
          "Feeling worried about the upcoming exams. So much to study, so little time. 😰 #StressedOut #ExamSeason #Nervous": 'worry',
          "Just heard about the recent spike in COVID cases. It's concerning, and we need to take all necessary precautions. Stay safe, everyone! 🙏 #COVID19 #StaySafe #Precautions": 'worry'
}


In [ ]:
preprocessed_tweets = [ clean_content(element) for element in test_tweets.keys()]
labels = [ element for element in test_tweets.values()]

In [ ]:
preprocessed_tweets

['believ disrespect peopl infuri angri frustrat outrag',
 'worst custom servic experi ever feel furiou right angrycustom unbelievableservic',
 'lucki found soulmat everi day feel like dream come true love grate bless',
 'love beauti feel world make everyth brighter better spread love everywher go spreadlov loveisallyoune',
 'finish read interest book thoughtprovok journey booklov thought recommend',
 'weather today neither hot cold perfect walk park weatherupd enjoyingnatur',
 'woke feel incred motiv readi conquer day let u make product one folk motiv positivevib gogetit',
 'receiv great news today hard work final paid feel top world right achievementunlock happyd',
 'feel heartbroken today hard let go someth held dear heartbroken emot sad',
 'sometim life throw curvebal okay feel sad rememb take one step time lean love one support sadbutstrong supportsystem',
 'omg got amaz surpris friend truli know make feel special surpris bless friendsforev',
 'expect find hidden gem restaur small 

In [ ]:
labels

['anger',
 'anger',
 'love',
 'love',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'sadness',
 'sadness',
 'surprise',
 'surprise',
 'worry',
 'worry']

In [ ]:
models = ['cogitosum84/twittersentimentv2',
          'TheJournal/higher_accuracy_Bert_Model',
          'LionelNH/tweetSentiment4',
          'pingyaol/model_demo']

In [62]:
import requests
headers = {"Authorization": "Bearer API_TOKEN"}

def query(API_URL,
          payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

In [63]:
for model in models:
  API_URL = "https://api-inference.huggingface.co/models/"+str(model)
  final_score = 0
  for index,tweet in enumerate(preprocessed_tweets):
    output = query(API_URL,
                  {"inputs": tweet}
                  )
    print(output)
    pred_labels = [item['label'] for item in output[0][:3]]
    if labels[index] in pred_labels:
      final_score += 1
  print(f"Final score of {model} is {final_score}")

[[{'label': 'surprise', 'score': 0.18476217985153198}, {'label': 'neutral', 'score': 0.16617737710475922}, {'label': 'worry', 'score': 0.14785362780094147}, {'label': 'positive', 'score': 0.14385366439819336}, {'label': 'anger', 'score': 0.11999209970235825}, {'label': 'love', 'score': 0.1193402037024498}, {'label': 'sadness', 'score': 0.11802073568105698}]]
[[{'label': 'sadness', 'score': 0.20953351259231567}, {'label': 'surprise', 'score': 0.17926213145256042}, {'label': 'worry', 'score': 0.15526065230369568}, {'label': 'positive', 'score': 0.1549469232559204}, {'label': 'neutral', 'score': 0.1093641072511673}, {'label': 'love', 'score': 0.10872558504343033}, {'label': 'anger', 'score': 0.08290708065032959}]]
[[{'label': 'surprise', 'score': 0.21866163611412048}, {'label': 'sadness', 'score': 0.2177686244249344}, {'label': 'neutral', 'score': 0.1478194147348404}, {'label': 'positive', 'score': 0.14738672971725464}, {'label': 'worry', 'score': 0.12642040848731995}, {'label': 'anger', 